In [2]:
import torch
import clip
from PIL import Image

# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
model, preprocess = clip.load("ViT-L/14", device=device)

c:\Users\onetaste108\.conda\envs\fapp\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
torch.cuda.get_device_name(0)


'NVIDIA GeForce RTX 2070 with Max-Q Design'

In [67]:
import os
import numpy as np

features_dir = '../weights'
fps = os.listdir(features_dir)

indices = [int(f.split('.')[0]) for f in fps]
indices.sort()

all_size = 0
features = []
for idx in indices:
	# if idx == 0: continue
	arr = np.load(features_dir+'/'+f'{idx}.npy')[0]
	arr = np.float32(arr)
	# print(f'{idx+512}.npy', arr[0][:5])
	# print([arr[i][0] for i in range(8)])
	all_size += arr.shape[0]
	features.append(arr)
	if idx == 0:
		features.append(np.ones_like(arr)*10.0)

features_list = features

print('all size', all_size)
print('supposed', indices[-1]+1024)

for f in features:
	assert f.shape[1] == 768, f.shape
features = np.concatenate(features, 0)

print('weights shape', features.shape)


fap_features = features
fap_features_ = torch.from_numpy(fap_features).to(device)
fap_features = fap_features_
fap_features /= fap_features.norm(dim=-1, keepdim=True)


def find_best_feature(image_features, fap_features, size=10):
	image_features /= image_features.norm(dim=-1, keepdim=True)
	similarity = (100.0 * image_features @ fap_features.T).softmax(dim=-1)
	values, best = similarity[0].topk(size)
	values = values.cpu().numpy()
	best = best.cpu().numpy()
	return values, best

def sim_image(image, fap_features, size=10):
	image = preprocess(image).unsqueeze(0).to(device)
	with torch.no_grad():
		image_features = model.encode_image(image)
	return find_best_feature(image_features, fap_features, size)

def sim_text(text, fap_features, size=10):
	text = clip.tokenize([text]).to(device)
	with torch.no_grad():
		image_features = model.encode_text(text)
	return find_best_feature(image_features, fap_features, size)

def sim_db(id, fap_features, size=10):
	feat = fap_features[id:id+1]
	return find_best_feature(feat, fap_features, size)





with open('../index.txt', 'r') as f:
	index = f.read().split('\n')

from io import BytesIO
import requests
def load_image(url):
	response = requests.get(url)
	assert url[-4:] == '.jpg', url
	img = Image.open(BytesIO(response.content))
	return img

def load_index(idx):
	url = index[idx]
	print('loading', url)
	url = 'https://my-project-420-353715.appspot.com.storage.googleapis.com/'+url
	img = load_image(url)
	return img

from IPython import display
def display_index(idx):
	display.display(load_index(idx))

all size 306176
supposed 306176
weights shape (307200, 768)


In [68]:
for i in range(0, len(features_list), 1):
	idx = i*1024
	img = load_index(idx)
	b = sim_image(img, fap_features, 10)[1][0]
	diff = idx - b
	print(diff, idx, b)

loading database/fapello/models/abella-danger/abella-danger_0001.jpg
0 0 0
loading database/fapello/models/abigaiil-morris/abigaiil-morris_0011.jpg
4 1024 1020
loading database/fapello/models/adelalinka-twins/adelalinka-twins_0048.jpg
0 2048 2048
loading database/fapello/models/aery-tiefling/aery-tiefling_0100.jpg
0 3072 3072
loading database/fapello/models/agos-ashford/agos-ashford_0057.jpg
0 4096 4096
loading database/fapello/models/ai-generated-girls-1/ai-generated-girls-1_0146.jpg
0 5120 5120
loading database/fapello/models/aida-cortes/aida-cortes_0039.jpg
0 6144 6144
loading database/fapello/models/aisultana/aisultana_0375.jpg
-822 7168 7990
loading database/fapello/models/aisux/aisux_0770.jpg
0 8192 8192
loading database/fapello/models/alanah-pearce/alanah-pearce_0424.jpg
0 9216 9216
loading database/fapello/models/alcololi/alcololi_0677.jpg
0 10240 10240
loading database/fapello/models/alena-witch/alena-witch_0349.jpg
0 11264 11264
loading database/fapello/models/alexa-breit/ale

KeyboardInterrupt: 

In [72]:
ref_id = 4600
ref = load_index(ref_id)
# ref = Image.open('../img/test1.jpg')
# display.display(ref)

# vb = sim_text('red dress', fap_features, 10)
# vb = sim_image(ref, fap_features, 10)
vb = sim_db(ref_id, fap_features, 10)
print(vb[1])

# for v, b in zip(*vb):
# 	idx = b if b < 1024 else b+1024
# 	print(idx)
# 	display_index(idx)
# 	print('Value:', v)


loading database/fapello/models/agos-ashford/agos-ashford_0561.jpg
[4600 4608 4596 4458 4598 4601 4607 4886 4597 4595]


In [35]:
import os
import numpy as np

features_dir = '../weights'
fps = os.listdir(features_dir)

indices = [int(f.split('.')[0]) for f in fps]
indices.sort()

all_size = 0
features = []
for idx in indices:
	arr = np.load(features_dir+'/'+f'{idx}.npy')[0]
	arr = np.float32(arr)
	# print(f'{idx+512}.npy', arr[0][:5])
	# print([arr[i][0] for i in range(8)])
	all_size += arr.shape[0]
	features.append(arr)

count = 0
for i in range(len(features)-1):
	if np.mean(np.abs(features[i] - features[i+1])) < 0.2:
		count += 1
		
print(count)

0


In [6]:
n = image_features.cpu().numpy()
n = np.array([n,n,n])
print(n.dtype)
np.save('test', n)
n2 = np.load('../weights/clip512.npy')
print(n2.dtype)

NameError: name 'image_features' is not defined

In [ ]:
test = [f,f]
np.array(test)

array([[[-0.1383 , -0.746  ,  0.296  , ...,  0.128  ,  0.05353,
         -0.4373 ]],

       [[-0.1383 , -0.746  ,  0.296  , ...,  0.128  ,  0.05353,
         -0.4373 ]]], dtype=float16)

In [7]:
import pickle
obj = [1,2]
with open("test.pkl", "w") as f:
	pickle.dump(obj, f)

TypeError: write() argument must be str, not bytes